# Preprocessing the CV features
 Installation of important libraries

In [2]:
!pip install xgboost
!pip install gensim

  Using cached gensim-4.2.0-cp39-cp39-win_amd64.whl (23.9 MB)
  Using cached Cython-0.29.28-py2.py3-none-any.whl (983 kB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\__pycache__\\cython.cpython-39.pyc'
Consider using the `--user` option or check the permissions.



In [9]:
%pip install wordcloud
from wordcloud import WordCloud

In [10]:
import re
import logging 
import pandas as pd
%matplotlib inline
import pandas as pd
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

Read the dataframe and observe the column features

In [77]:
path = 'C:\\Users\\sande\\Documents\\GitHub\\Project - Toroni NLP Instagram\\NLP-project-\\'
dfRaw = pd.read_csv(path+ "Dataset\\RAW_FINAL_DATAFRAME_88k_29042022.csv")
dfRaw.head(2)

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,image_type,account,caption,profile_name,profile_image_link,biography,id,external_url,following,...,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry,_category,region
0,0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.620408e+18,http://www.yeastbakery.com/,352.0,...,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...",0,"[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)",hospitality,bakery,UK
1,1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.790859e+18,http://www.yeastbakery.com/,352.0,...,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)",hospitality,bakery,UK


In [4]:
dfRaw.columns

Index(['Unnamed: 0', 'image_type', 'account', 'caption', 'profile_name',
       'profile_image_link', 'biography', 'id', 'external_url', 'following',
       'likes', 'media_type', 'posts_count', 'followers', 'is_verified',
       'datetime', 'image_url.1', 'url', 'comments', 'input_account',
       'input_posts_number', 'input_date_of_post', 'input_post_type',
       'input_stop_id', 'warning', 'error', 'tags', 'confidence_score',
       'accent_color', 'is_bw', 'dominant_colors', 'bg_color', 'fore_color',
       'industry', '_category', 'region'],
      dtype='object')

Seperating the kinds of data types to be dealt with

In [50]:
required_col_names = ['image_type','tags', 'confidence_score','accent_color', 'is_bw', 'dominant_colors', 'bg_color', 'fore_color' ] 
textual_col_names = ['image_type','tags']
numerical_col_names = ['confidence_score','accent_color', 'dominant_colors', 'bg_color', 'fore_color']

In [51]:
df = dfRaw[required_col_names]

In [52]:
#calculating missing values in the dataset
missing_values = df.isnull().sum()
missing_per = (missing_values/df.shape[0])*100
missing_table = pd.concat([missing_values,missing_per], axis=1, ignore_index=True) 
missing_table.rename(columns={0:'Total Missing Values',1:'Missing %'}, inplace=True)
missing_table

,Total Missing Values,Missing %
image_type,0,0.0
tags,0,0.0
confidence_score,0,0.0
accent_color,0,0.0
is_bw,0,0.0
dominant_colors,0,0.0
bg_color,0,0.0
fore_color,0,0.0


Exploring columns now:

In [55]:
df.head(2)

,image_type,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,"['Ambiguous Clip', 'Not LineDraw']","['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...",0,"[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)"
1,"['Not Clip', 'Not LineDraw']","['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)"


In [56]:
df['image_type'].value_counts()

['Not Clip', 'Not LineDraw']          79660
['Ambiguous Clip', 'Not LineDraw']     6318
['Good Clip', 'Not LineDraw']          1397
['Normal Clip', 'Not LineDraw']         921
['Not Clip', 'LineDraw']                122
['Ambiguous Clip', 'LineDraw']           57
['Good Clip', 'LineDraw']                31
['Normal Clip', 'LineDraw']              10
Name: image_type, dtype: int64

Applying preprocessing on the textual columns now:

In [58]:
def findall_re(val, re2apply):
    v = re2apply.findall(val)
    summarised = []
    for i in v:
        summarised.append(i[1])
    return summarised
    
re_textual = re.compile(r"([\"'])(.*?[^\\])\1")
for text_col in textual_col_names:
    for i in df.iterrows():
        y = findall_re(df.at[i[0],text_col], re_textual)
        df.at[i[0], text_col] = y

df.head(2)


,image_type,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,"[Ambiguous Clip, Not LineDraw]","[text, font, graphics, screenshot, graphic des...","[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...",0,"[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)"
1,"[Not Clip, Not LineDraw]","[dessert, baked goods, baking, snack, food, sw...","[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)"


In [63]:
for i in textual_col_names:
    print(type(df.at[0,i]))

<class 'list'>
<class 'list'>


Handling numerical columns:

In [66]:
for i in numerical_col_names:
     print("The type of column", i, "is :", type(df.at[0,i]))

The type of column confidence_score is : <class 'str'>
The type of column accent_color is : <class 'str'>
The type of column dominant_colors is : <class 'str'>
The type of column bg_color is : <class 'str'>
The type of column fore_color is : <class 'str'>


In [67]:
df[numerical_col_names].head(2)

,confidence_score,accent_color,dominant_colors,bg_color,fore_color
0,"[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...","[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)"
1,"[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...","[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)"


In [68]:
re_removes_brackets = re.compile('[0-9][^)]+[0-9]')
def convert_str2float_re(i):
  n = []
  y = i.split(', ')
  if len(y[0]) == 0:
    return []
  for j in range(len(y)):
    n.append(float(y[j]))
  return n

for col_name in numerical_col_names:
  print(col_name)
  for i in df.iterrows():
    num_val = df.at[i[0], col_name]
    if len(num_val) == 2:
      df.at[i[0], col_name] = [0]
    else:
      num_val = re_removes_brackets.findall(num_val)
      df.at[i[0], col_name] = convert_str2float_re(num_val[0])

df[numerical_col_names].head(2)

confidence_score
accent_color
dominant_colors
bg_color
fore_color


,confidence_score,accent_color,dominant_colors,bg_color,fore_color
0,"[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]"
1,"[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]"


In [73]:
for i in numerical_col_names:
     # print(df.at[1,i])
     print("The type of column", i, "is :", type(df.at[1,i]))
     pass
     # print("Type for {}:".format(i),type(df.at[0,i]), type(df.at[0,i][0]))

The type of column confidence_score is : <class 'list'>
The type of column accent_color is : <class 'list'>
The type of column dominant_colors is : <class 'list'>
The type of column bg_color is : <class 'list'>
The type of column fore_color is : <class 'list'>


Dataframe obtained with updated CV features

In [74]:
df.head(2)

,image_type,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,"[Ambiguous Clip, Not LineDraw]","[text, font, graphics, screenshot, graphic des...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]"
1,"[Not Clip, Not LineDraw]","[dessert, baked goods, baking, snack, food, sw...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]"


In [75]:
for i in df.columns:
     print(type(df.at[0,i]))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'numpy.int64'>
<class 'list'>
<class 'list'>
<class 'list'>


In [80]:
dfRaw.head(2)

,Unnamed: 0,image_type,account,caption,profile_name,profile_image_link,biography,id,external_url,following,...,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry,_category,region
0,0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.620408e+18,http://www.yeastbakery.com/,352.0,...,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...",0,"[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)",hospitality,bakery,UK
1,1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.790859e+18,http://www.yeastbakery.com/,352.0,...,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)",hospitality,bakery,UK


In [82]:
#datatype of CV features in original df
for i in df.columns:
     print(type(dfRaw.at[0,i]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'str'>


Replace these updated columns in the parent dataframe

In [92]:
for i in df.columns:
    dfRaw[i] = df[i]
    print("The updated type of column", i, "is :",type(dfRaw.at[0,i]))

The updated type of column image_type is : <class 'list'>
The updated type of column tags is : <class 'list'>
The updated type of column confidence_score is : <class 'list'>
The updated type of column accent_color is : <class 'list'>
The updated type of column is_bw is : <class 'numpy.int64'>
The updated type of column dominant_colors is : <class 'list'>
The updated type of column bg_color is : <class 'list'>
The updated type of column fore_color is : <class 'list'>


In [85]:
dfRaw.to_csv('data.csv')